# Import av ISEE kontroller til Edith

- Må ha laget rådata tabell først
- Må velge felter som skal være med

In [ ]:
import getpass # Denne funksjonen lar deg skrive inn passordet med et input felt, istedenfor å skrive det inn i notebooken
passord = getpass.getpass()

### Felter som inkluderes

In [ ]:
inkluderte_felter = [
    'feilliste',
    'NAVN',
    'ORGNR',
    'FELT_ID',
    'KOMMUNE',
    'FYLKE',
    'Stratum',
    'Driftsform',
    'DELREG_NR_x',
    'ENHETS_TYPE_x',
    'ENHETS_ID',
    'SKJEMA_x',
    'LOPENR_x',
    'RAD_NR_x',
    'index',
    'RAD_NR_y',
    'LOPENR_y',
    'SKJEMA_y',
    'FEILID',
    'VARIABEL',
    'DELREG_NR_y',
    'År_2021',
    'ORG_FORM',
    'NACE1',
    'ENHETS_TYPE_y',
    'orgnrNavn',
    'kommentar']

In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import Table, create_engine
from sqlalchemy.sql import select
from flask_sqlalchemy import SQLAlchemy
import getpass
import cx_Oracle
import json
import os

from models.models_delt import connect

In [ ]:
with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

conn, engine, db = connect()

In [ ]:
path = "/ssb/stamme02/jordbruk/sblbrund/wk8"
conn = sqlite3.connect(f"{path}/edith.sqlite", timeout=15) #Må endres hvis koblingen skal være opp mot dynarev
engine = create_engine(f"sqlite:///{path}/edith.sqlite")
bruker = getpass.getuser()[:3]
db = cx_Oracle.connect(bruker,passord, "DB1P")

In [ ]:
df_feil = pd.read_sql(f"SELECT * FROM DYNAREV.VW_KONTROLL_FEIL WHERE DELREG_NR='63021'", con = db)
# Nedenfor fjernes de som er reviderte
df_feil = df_feil.loc[df_feil["ENHETS_ID"].isin(list(pd.read_sql(f"SELECT ENHETS_ID, REV_KODE, INN_DATO, REV_DATO FROM DYNAREV.VW_SKJEMA_ENHET WHERE DELREG_NR = 63021 and REV_KODE = 'U'", con=db)["ENHETS_ID"].unique()))]

In [ ]:
dff = pd.read_sql(f'SELECT * FROM {config["tabeller"]["raadata"]}', con=engine)

In [ ]:
pd.read_sql(f"SELECT * FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = 63021 AND AKTIV=1 ", con = db)

In [ ]:
df = pd.merge(df_feil, dff, on= "ENHETS_ID", how = "right")

In [ ]:
df["kommentar"] = ""
df["feilliste"] = df["FTEKST"]

In [ ]:
df = df.dropna(subset = ["feilliste"])
df = df.drop_duplicates(subset = ["ORGNR", "feilliste", "FELT_ID"])

In [ ]:
df[inkluderte_felter].to_csv(f"{config['data']['filsti']}/feilliste.csv", index = False)

In [ ]:
for i in df_feil["FTEKST"].unique():
    print(i)
    print("---------")